# Neo4j Python API Practice (with Simple Transactions)
### Overview:

Teresa will write her own Jupyter Notebook where...
- She will access a Neo4j database (local)
- She will create simple nodes and relationshps and write them to the database
- She will read from the database
- Documentation she is using at this [link](https://neo4j.com/docs/python-manual/current/).  See also this [link](https://neo4j.com/docs/api/python-driver/current/api.html), but it is more confusing
  
### Motivation:
- Become familiar with Neo4j Python API
- Put together the beginnings of a "Touch-less Data Management System" for the Buonassisi Lab
- Continue practicing with python language and Jupyter Notebook

## Step 0:

If you are attempting to run this code, you will first need to start your own local DBMS using the [Neo4j Desktop App](https://neo4j.com/download/?utm_source=Google&utm_medium=PaidSearch&utm_campaign=Evergreen&utm_content=AMS-Search-SEMBrand-Evergreen-None-SEM-SEM-NonABM&utm_term=download%20neo4j&utm_adgroup=download&gad_source=1&gclid=Cj0KCQjwpNuyBhCuARIsANJqL9Mfw2KSzysHnaaX0w_SPaPP49aDQPg5k6T-joWu_UnTcMYiWsrE4NEaAm4TEALw_wcB).

The default Bolt Port that is used should be 7687, but you can change the code to match your settings.

## Step 1: Load the required packages, as usual:

In [140]:
from neo4j import GraphDatabase
import pandas as pd

## Step 2: Connect to a database:

### Establish the DRIVER

In [59]:
# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "neo4j://localhost:7687" # Specify URI of already running database
AUTH = ("neo4j","thisispractice") # Enter username and password (this should probably be a reqeusted input in final product)

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

## Step 3: Create node(s) and relationships(s) in the database:

In [36]:
# Create constraints first so that I don't duplicate nodes when running this practice.
# Apparently: One needs to run these separately if using a driver (which we are), but you can run them in one query if using Neo4j Browser if you separate with ;.
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query("""
                    CREATE CONSTRAINT `name person_uniq` IF NOT EXISTS
                    FOR (n: `Person`)
                    REQUIRE (n.`name`) IS UNIQUE;
                     """,
                     database = "neo4j")
    driver.execute_query("""
                    CREATE CONSTRAINT `name dog_uniq` IF NOT EXISTS
                    FOR (d: `Dog`)
                    REQUIRE (d.`name`) IS UNIQUE
                     """,
                     database = "neo4j") # it was recommended to always specify the database for performancy optimization, standard configuration is one main database called "neo4j"
    driver.close() #closing the driver for security reasons, but opening and closing a connection is a costly operation. You should not do this after every query in your final product.


In [37]:
# Create 2 nodes (Person) and (Dog) where Person -LOVES-> Dog
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    # MERGE is a combination of CREATE and MATCH
    # Using this allows me to overwrite data rather than create new nodes if I run this code more than once.
    driver.execute_query("""
                         MERGE (p:Person {name: "Teresa"}) // Create a person node with name property = "Teresa"
                         SET p.DOB = date("0001-01-01") // Set the DOB property of the person to "0001-01-01
                         MERGE (d:Dog {name: "Snow"}) // Create a dog node with name property = "Snow"
                         SET d.Breed = "Mutt" // Set the Breed property of the dog to "Mutt"
                         MERGE (p) -[:LOVES]-> (d) // Create the relationship (person named Teresa) -[LOVES]-> (dog named Snow) """,
                         database = "neo4j")
    driver.close()
                         

## Step 4: Query (Read) from the database

Note: I already checked that the creation step worked by having Neo4j Bloom opened simultaneously.  This section is more to demonstrate how to query (and have it printed/displayed) through python.

In order to have the results of a query returned in python, one must save the records, summary, keys of the driver.execute_query() function.

### Creating some new nodes just to make the outputs more interesting

In [41]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query("""
                         MERGE (michael:Person {name: "Michael"})
                         SET michael.DOB = date("0010-01-01")
                         MERGE (tracy:Person {name: "Tracy"})
                         SET tracy.DOB= date("0002-01-01")
                         WITH tracy, michael
                            MATCH (d:Dog {name: "Snow"})
                            MERGE (tracy) -[:LOVES]-> (d)
                            MERGE (d) -[:HATES]-> (michael) """,
                         database = "neo4j")
    driver.close()

### Now actually executing the query

In [141]:
# Query
with GraphDatabase.driver(URI, auth=AUTH) as driver:
   records, summary, keys = driver.execute_query(
    "MATCH (p:Person) RETURN p.name AS name, p.DOB as DOB",
    database_="neo4j")
driver.close()

# Loop through results and do something with them
#for record in records:
#    print(record)  # obtain returned persons and all info as list
for record in records: 
    print(record.data()) #obtain record as dict

# Summary information
print("The query `{query}` returned {records_count} records in {time} ms.".format(
    query=summary.query, records_count=len(records),
    time=summary.result_available_after
))

print(f"Available keys are {keys}")  # ['name', 'DOB']

{'name': 'Teresa', 'DOB': neo4j.time.Date(1, 1, 1)}
{'name': 'Tracy', 'DOB': neo4j.time.Date(2, 1, 1)}
{'name': 'Michael', 'DOB': neo4j.time.Date(10, 1, 1)}
The query `MATCH (p:Person) RETURN p.name AS name, p.DOB as DOB` returned 3 records in 1 ms.
Available keys are ['name', 'DOB']


### Return as a pandas table for easier viewing and exporting of data

In [139]:
records_list = []
for record in records:
    records_list.append(record.data())
pd.DataFrame(records_list)

,name,DOB
0,Teresa,0001-01-01
1,Tracy,0002-01-01
2,Michael,0010-01-01
